In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator
from sklearn.datasets import make_regression
from sklearn.datasets import make_friedman1

In [21]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in xrange(self.iters):
            y_pred = np.array(y_data) + curr_pred
            algo = DecisionTreeRegressor(self.tree_params_dict)
            algo.fit(X_data, y_pred)
            curr_pred = curr_pred - algo.predict(X_data)
            self.estimators.append(algo)
    
    def predict(self, X_data):
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            res += estimator.predict(X_data)
        return res

## Проверка качества полученного класса

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

In [23]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [24]:
algo.fit(X_data, y_data)

TypeError: unhashable type: 'dict'

Сгенерируем данные с линейной зависимостью

In [22]:
X_data, y_data = make_regression(n_samples=1000, noise=10, n_features=10, random_state=42)

In [19]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [20]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

TypeError: unhashable type: 'dict'

In [6]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=4), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-7550.7275207710281

In [7]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-95.517114056519787

на реально-линейной зависимости не дотягивает до линейной модели, но уже существенно лучше чем одно дерево.

И аналогично, сгенерируем нелинейную зависимость

In [8]:
X_data, y_data = make_friedman1(n_samples=1000, noise=10, n_features=10, random_state=42)

In [9]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth': 1
    },
    iters=100,
    tau = 0.1
)

In [10]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-105.81605332375698

In [11]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=6), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-139.62482310819027

In [12]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-108.80318041227224

Уже получилось лучше линейной модели